In [11]:
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
import src.trainers as tr
import importlib as imp
import numpy as np

In [15]:
read_features_targets_kwargs = {'fields_to_read' :    {"B": True,"B_ext": False,"divB": False,"E": True,"E_ext": False,"rho": True,"J": True,
                                                      "P": True,"PI": True,"Heat_flux": False,"N": False,"Qrem": False}, # which fields to read
                                'request_features' :  ['rho_e', 'Bx', 'By', 'Bz', 'Vx_e', 'Vy_e', 'Vz_e', 'Ex', 'Ey', 'Ez'],  # what network needs to make predictions
                                'request_targets' :   ["Pxx_e", "Pyy_e","Pzz_e","Pxy_e","Pxz_e","Pyz_e"],                     # what we want to predict
                                'choose_species' :    ['e',None],   # which species to take
                                'choose_x' : [0,64], 'choose_y' : [0,64], 'verbose' : False  # which part of the domain to take. The full domain is 2048x2048 and is too heavy for the tutorial
                          } # make sure not to put a comma here, otherwise the object becomes a tuple rather than a dictionary which causes all sorts of problems

In [ ]:
dataset_kwargs = {'data_folder': "/volume1/scratch/share_dir/nn/data/raw_data/",               # where the data is stored
                          'image_file_name_column' : 'filenames',                       # the column in the csv file that contains the image file names
                          'read_features_targets_kwargs' : read_features_targets_kwargs,
                          'train_sample' :  "/volume1/scratch/share_dir/ecsim/sampling/peppe/intraexp/train.csv",  # this should be the path to the csv file that contains a list of training samples, each cell corresponds to local paths to the actual data relative to `data_folder`
                          'val_sample' :    "/volume1/scratch/share_dir/ecsim/sampling/peppe/intraexp/val.csv",    # same but validation
                          'test_sample' :   "/volume1/scratch/share_dir/ecsim/sampling/peppe/intraexp/test.csv",  # same but testing
                          'scaler_features' : False, #True,    # Whether to normalize the features. It is called scaler because it rescales to 0 mean and 1 variance
                          'scaler_targets' : False, #True,     # Whether to normalize the targets
                          'prescaler_targets' : None #['log','log','log',None,None,None]  # If we want to transform the targets before training. The order in which they appear correspond to `request_targets` in `read_features_targets_kwargs``
                      }

In [17]:
import pandas as pd
pd.read_csv(dataset_kwargs['test_sample'])

,filenames
0,high_res_hbg/DoubleHarris-Fields_011000.h5
1,high_res_hbg/DoubleHarris-Fields_014000.h5
2,high_res_hbg/DoubleHarris-Fields_019000.h5


In [18]:
load_data_kwargs={'train_loader_kwargs' : {'batch_size' : 512,      #training batch size
                                           'subsample_rate' : .2,   #subsample rate which defines a fraction of data that is used when sampling. 1 means all data is used
                                           'subsample_seed' : 42,   #seed for the subsampling
                                           'seed' : 42,             #seed for the random number generator
                                           'shuffle' : True},       #shuffle the training set
                    'val_loader_kwargs' : {'batch_size' : 512,        #validation batch size
                                           'subsample_rate' : .2, 
                                           'subsample_seed' : 42, 
                                           'shuffle' : False}}      # we don't need to shuffle the validation set

In [19]:
model_kwargs={'model': 'MLP',    # the model class name we want to use, check `src/models.py` for available models
              'model_seed' : 42, # seed for the random initialization of the model
              'feature_dims' : [10, 60, 80, 50, 40, 6], # the dimensions of the layers in the model. The first element should be the number of `request_features` in `read_features_targets_kwargs` and the last element should be the number of `request_targets` in `read_features_targets_kwargs`
              'activations' : ['Tanh','Tanh','ReLU','Tanh', None], 
    'optimizer_kwargs': {'optimizer' : 'Adam',      # look up https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#adam 
                         'lr' : 5e-4,               # learning rate
                         'criterion': 'MSELoss'},   # the loss function to optimize
    'scheduler_kwargs': {'scheduler' : 'ReduceLROnPlateau',  # look up https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html
                         'mode' : 'min',  # the remainder are the arguments for the ReduceLROnPlateau scheduler
                         'factor' : 0.2, 
                         'patience' : 10, 
                         'cooldown' : 15 , 
                         'epochs' : 10,                 # number of epochs before stoppig training
                         'early_stopping' : 20},        # number of epochs to tolarate without 
    'logger_kwargs':{'update_step': 1, 'show': True}}   # some details about how to log the training process

In [20]:
work_dir = '/volume1/scratch/georgem/closure/dev/dev16/'  # you may have to create containing path before running the script (check in the source code). It is better to use absolute paths than relative paths for the operations below to work
                                                                            # the /dev16/ folder on the other hand will be definitely created by the trainer so you don't need to create it
! rm -rf {work_dir}  # This will delete the previously existing model
trainer = tr.Trainer(work_dir=work_dir,   # where the model will be saved
                     dataset_kwargs=dataset_kwargs, load_data_kwargs=load_data_kwargs, model_kwargs=model_kwargs)

2025-07-14 16:07:04,187 | INFO | src.trainers | 	 Creating a new configuration file: /volume1/scratch/georgem/closure/dev/dev16/config.json
2025-07-14 16:07:04,188 | INFO | src.trainers | 	  
2025-07-14 16:07:04,189 | INFO | src.trainers | 	 ===Logging to /volume1/scratch/georgem/closure/dev/dev16//training.log on level 20, @ self.rank=None, self.local_rank=None, self.device=device(type='cpu') ===
2025-07-14 16:07:04,189 | INFO | src.trainers | 	 host: haydn.esat.kuleuven.be
2025-07-14 16:07:04,190 | INFO | src.trainers | 	  
2025-07-14 16:07:04,191 | INFO | src.datasets | 	  This is train set
2025-07-14 16:07:04,191 | INFO | src.datasets | 	  This is train set
2025-07-14 16:07:04,192 | INFO | src.datasets | 	 Datasplit performed according to /volume1/scratch/share_dir/ecsim/sampling/peppe/intraexp/train.csv
2025-07-14 16:07:04,192 | INFO | src.datasets | 	 Datasplit performed according to /volume1/scratch/share_dir/ecsim/sampling/peppe/intraexp/train.csv
2025-07-14 16:07:05,986 | INFO

In [26]:
import torch
torch.mean(trainer.train_dataset.features)

tensor(2.9802e-09)

In [27]:
! ls {work_dir}

config.json  training.log  X.pkl  y.pkl


In [28]:
print(f"trainer work dir: \t\t{trainer.work_dir}")
print(f"trainer class instance: \t{trainer}")
print(f"trainer loader: \t\t{trainer.train_loader}")
print(f"train dataset features shape: \t {trainer.train_dataset.features.shape}")
print(f"train dataset targets shape: \t{trainer.train_dataset.targets.shape}")

trainer work dir: 		/volume1/scratch/georgem/closure/dev/dev16/
trainer class instance: 	<src.trainers.Trainer object at 0x7f4805563a10>
trainer loader: 		<src.datasets.ChannelDataLoader object at 0x7f480592c920>
train dataset features shape: 	 torch.Size([32768, 10])
train dataset targets shape: 	torch.Size([32768, 6])


In [29]:
trainer.fit()

2025-07-14 16:08:08,170 | INFO | src.trainers | 	 Prior to fit: RAM memory % used: 3.8, RAM Used (GB):, 8.257073152, process RAM usage (GB): 0.6456718444824219
2025-07-14 16:08:08,170 | INFO | src.trainers | 	 Prior to fit: RAM memory % used: 3.8, RAM Used (GB):, 8.257073152, process RAM usage (GB): 0.6456718444824219
2025-07-14 16:08:08,172 | INFO | src.models | 	 Each forward pass had 13 train batches.
2025-07-14 16:08:08,172 | INFO | src.models | 	 Each forward pass had 13 train batches.
2025-07-14 16:08:09,719 | INFO | src.models | 	 Number of samples per batch: len(next(iter(train_loader))[0]) = 512
2025-07-14 16:08:09,719 | INFO | src.models | 	 Number of samples per batch: len(next(iter(train_loader))[0]) = 512
2025-07-14 16:08:14,419 | INFO | src.models | 	 Epoch 1/10 | Train loss: nan | Val loss: nan | Time/epoch: 4.698 s | Time/epoch_train: 3.209 s | Learn rate: [0.0005]
2025-07-14 16:08:14,419 | INFO | src.models | 	 Epoch 1/10 | Train loss: nan | Val loss: nan | Time/epoch:

inf

In [37]:
trainer.train_dataset.targets.shape

torch.Size([32768, 6])